# Analysis: Topic Models

### Import required libraries

In [ ]:
import pandas as pd
import sys
import numpy as np
import csv
import nltk
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from IPython.display import display
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.sklearn
import pickle
import xml.etree.ElementTree as ET
import glob
import re

### Import CSVs
* Import df-R-cleaned.csv and df-n.csv from the output of `2_clean.ipynb`. 
* Then combine both into a new dataframe to use in this session, 'df_all', matching on n_id and id.

In [ ]:
df_R = pd.read_csv('output/df-R-cleaned.csv', encoding='utf-8', na_filter=False) #article metadata
df_n = pd.read_csv('output/df-n.csv', encoding='utf-8') #article ngrams
df_all = df_R.merge(df_n, left_on='file_name', right_on='n_id')

### Check on number of articles for analysis
Ensure we are excluding the records we previously removed from df_R.

In [ ]:
assert len(df_R) < len(df_n) 
assert len(df_all) == len(df_R) 
len(df_all)

### Corpus metadata: Descriptive Statistics
Run the numbers here on the basic outlines of the corpus; number of articles per year, per journal, etc.

In [ ]:
%matplotlib inline

#### Articles by year
Calculate and plot articles by year.

In [ ]:
year_count = df_R.groupby(['pub_year']).count()[['file_name']]
year_count.columns = ['article_count']
year_count.to_csv('output/articles_per_year.csv', encoding='utf-8', index=True, header=True)

In [ ]:
x_year = year_count.index
y_count = year_count['article_count']
plt.plot(x_year, y_count)
plt.ylabel("Number of articles")
plt.xlabel("Year")
plt.title('LQ Corpus: Number of articles per year')
#plt.show()
plt.savefig('output/lq_tm/plots/art_per_year.png')

#### Issues by year

In [ ]:
journals = df_R.groupby(['pub_year', 'volume', 'issue']).count()[['file_name']]
journals.columns = ['article_count']
journals = journals.sort_values('pub_year', ascending=True)
journals
journals.to_csv('output/articles_per_issue.csv', encoding='utf-8', index=True, header=True)

In [ ]:
# number of entries = number of journal issues
journals.info()

In [ ]:
print('Mean articles per issue:', journals.article_count.mean()) # cody's number: 22.728070175438596 my number: 23.681286549707604

### Mean articles per issue

In [ ]:
journals = df_R.groupby(['pub_year']).count()[['file_name']]
journals.columns = ['article_count']
journals = journals.sort_values('pub_year', ascending=True)
journals.article_count.mean() # cody's number: 91.44705882352942 my number: 95.28235294117647
journals.to_csv('output/background/articles_per_year.csv', encoding='utf-8', index=True, header=True)

In [ ]:
mean_articles_issue = journals.groupby(['pub_year']).mean()[['article_count']]
mean_articles_issue.to_csv('output/background/mean_articles_per_issue.csv', encoding='utf-8', index=True, header=True)

In [ ]:
x_year = mean_articles_issue.index
y_count = mean_articles_issue['article_count']
plt.plot(x_year, y_count)
plt.ylabel("Articles per issue")
plt.xlabel("Year")
plt.title('LQ Corpus: Mean articles per issue')
#plt.show()
plt.savefig('output/lq_tm/plots/mean_art_per_issue.png')

### Article counts by type

In [ ]:
type_count = df_R.groupby(['article_type']).count()[['file_name']]
type_count.columns = ['type_count']
type_count = type_count.sort_values('type_count',ascending=False)
type_count.to_csv('output/article_types.csv', encoding='utf-8', index=True, header=True)

### Articles per journal title 
Not relevant for LQ study. But helpful whene working with multiple journal titles.

In [ ]:
journals_t = df_R.groupby(['journal_title']).count()[['file_name']]
journals_t.columns = ['journal_count']
journals_t = journals_t.sort_values('journal_count', ascending=False)
journals_t.to_csv('output/articles_per_journal_title.csv', encoding='utf-8', index=True, header=True)

### Overall wordcount 

In [ ]:
word_count = df_all['body'].apply(lambda x: len(str(x).split()))
word_count.sum() 

### CountVectorizer & LDA Topic Model
* Convert wordlists from df_all.body to Term Frequency vector. 
* tf = term frequency vector
* lda = latent dirichlet allocation; fit using tf model

In [ ]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.70, min_df=0.10,
                                max_features=None)

tf = tf_vectorizer.fit_transform(df_all.body.values.astype('U'))

### Create list of the stop words cut off by max_ and min_df thresholds: 
* max_df removes words that appear in more than 70% of articles and min_df removes words that appear in fewer than 10%
* max_df removes 24 words
* min_df removes 145,639  words

To document the words that have been removed from our dataset:

In [ ]:
with open('output/stop_words_all.txt', 'w') as f:
    for item in tf_vectorizer.stop_words_:
        f.write("%s\n" % item)

### LDA
Run LDA topic model. (Note: this cell may take 10+ minutes.)

In [ ]:
n_components = 40 # set the number of topics based on GridSearchCV best model NOTE: n_topics is deprecated

print("Fitting LDA models with tf features, "
      "n_components=%d..."
      % (n_components))

#define the lda function, with desired options
lda = LatentDirichletAllocation(n_components=n_components, max_iter=20, # number of iterations receommended by GredSearchCV best model ()
                                learning_method='online',
                                random_state=0)
#fit the model
lda.fit(tf)

#### Save the LDA model to a pickle file

In [ ]:
with open('output/lda_model.pk', 'wb') as pickle_file:
    pickle.dump(lda, pickle_file)

#### Reload LDA model from pickle file
You can start from the next cell in this notebook if you've previously run the LDA model.

In [ ]:
with open('output/lda_model.pk', 'rb') as pickle_file:
    lda = pickle.load(pickle_file)

In [ ]:
print("Log Likelihood: ", lda.score(tf))
print("Perplexity: ", lda.perplexity(tf))

### Top words per topic
Function to return the top words for each topic.

In [ ]:
n_top_words = 30 # how many words per topic
topic_word_list = []
def return_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        #topic_n = "\nTopic #%d:" % topic_idx)
        top_words = " ".join([feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]])
        topic_word_list.append(top_words) 
    topic_df = pd.DataFrame(topic_word_list)
    topic_df.to_csv('output/topic_words_lq_only.csv', encoding='utf-8', index=True, header=True)
    return(topic_df)

`topic_word_list[x]` will return the top 30 words for each topic. 

Save a list of the top words per topic to CSV, using the function defined above:

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
return_top_words(lda, tf_feature_names, n_top_words) 

### Most representative articles per topic
`article_list[x]` will return the top 10 articles aligned with each topic.

In [ ]:
topic_dist = lda.transform(tf)
topic_dist_df = pd.DataFrame(topic_dist)
df_w_topics = topic_dist_df.join(df_all)

`d` in the loop below is a counter that = topic number. 
It matches on the name of the topic columns (0, 1, 2...) from df_w_topics. Those columns (e.g., `df_w_topics[0]`) hold prevalence values for each article for each topic. The loop below transfers the articles with the highest ten prevalence values for each article to an article_list dataframe with key article metadata.

In [ ]:
article_list = []
for d in range(n_components): 
    tup = df_w_topics[['article_jcode', 'article_title', 'article_type', 'journal_title', 'journal_jcode', 'pub_year', d]].sort_values(by=[d], ascending=False)[0:10]
    #print(f'{d}: {tup}')
    article_list.append(tup)
    article_list[d].columns = ['article_jcode', 'article_title', 'article_type', 'journal_title', 'journal_jcode', 'pub_year', 'prevalence']

Flatten the list of article_list dataframes into a single dataframe adding a column with the topic number for each article.

In [ ]:
for n, i in enumerate(article_list):
    i['topic'] = n
article_lists_df = pd.concat(article_list)

#### Add book review titles
Find the titles of book reviews in the JSTOR XML since they aren't included in the R metadata (notebook 1) and append those titles to the top articles list.


In [ ]:
review_list = []
for index, row in article_lists_df[article_lists_df['article_title']==''].iterrows():
    #insert jstor code to pull book name and author from XML
    review_list.append(row['article_jcode'])
reviews_df_list = []
for xml_file in glob.iglob("metadata/*.xml"):
    jid = re.search(r'(\d*).xml$', xml_file).group(1) # jid (formerly xid) is the jstor code, it seems
    if jid not in review_list:
        continue
    tree = ET.parse(xml_file)
    for book_reviewed_node in tree.getroot().findall('./front/article-meta/product/source'):
        tup = (jid, book_reviewed_node.text)
        print(tup)
        reviews_df_list.append(tup)

reviews_df = pd.DataFrame(reviews_df_list, columns=['jid', 'book_reviewed'])

article_lists_df = article_lists_df.merge(reviews_df, how='outer', left_on='article_jcode', right_on='jid')
article_lists_df = article_lists_df.drop(['jid'], axis=1)
article_lists_df.loc[article_lists_df['article_type']=='book-review', 'article_title'] = article_lists_df['book_reviewed']
article_lists_df = article_lists_df.drop(['book_reviewed'], axis=1)
article_lists_df = article_lists_df.sort_values(by=['topic', 'prevalence'], ascending=[True, False])

Remove any duplicate rows and save the top articles to CSV for closer analysis.

In [ ]:
article_lists_df.drop_duplicates(inplace = True)

In [ ]:
article_lists_df.to_csv('output/top_articles_per_topic_lq.csv', encoding='utf-8', index=False, header=True)

Optional alternative: print output to multiple CSV files (one for each topic).

In [ ]:
for index, df in enumerate(article_list):
    filename = 'output/articles_per_topic/top_articles_' + str(index) + '.csv'
    df.to_csv(filename, encoding='utf-8', index=True, header=True)

### Map topics over time

First calculate word count per article and multiply topic weight by word count.

In [ ]:
df_w_topics['word_count'] = df_w_topics['body'].apply(lambda x: len(str(x).split()))

In [ ]:
col_list = []
for num in range(n_components): # This was originally called topic_columns. Not sure why yet... --naughton
    col = "%d_wc" % num
    col_list.append(col)
    df_w_topics[col] = df_w_topics[num] * df_w_topics['word_count']

#### Prevalence per topic
`e` in the loop below is a counter that = topic number + '\_wc'. It matches on the name of the topic columns (0_wc, 1_wc...) from df_w_topics. Those columns (e.g., df_w_topics[0_wc]) hold topic weights relative to word counts for each article.

In [ ]:
prevalence = []
for index, e in enumerate(col_list): 
    prev = df_w_topics[e].sum()/df_w_topics['word_count'].sum()
    tup = (index,prev)
    prevalence.append(tup)
prevalence = pd.DataFrame(prevalence)
prevalence.columns = ['topic','prevalence']
prevalence.sort_values(by=['prevalence'], ascending=False)

In [ ]:
prevalence.to_csv('output/prevalence_per_topic_lq.csv', encoding='utf-8', index=False, header=True)

Map prevalence of each topic by publication year.

In [ ]:
grouped_year = df_w_topics.groupby('pub_year')
fig3 = plt.figure()
# divide the number of topic words in each year by the total word count per year (so the figure adjusts to each year's output)
for e in col_list:
    ax2 = fig3.add_subplot(1,1,1)
    (grouped_year[e].sum()/grouped_year['word_count'].sum()).plot(kind='line', title=e)
    fig3.tight_layout()
    #plt.show()
    filename = 'output/plots/plot_' + str(e) + '.png'
    plt.savefig(filename)
    plt.close()

### Document term matrix
Create document-term-matrix dataframe, dtm_df, to look at the following (ignoring topic models):
* most common words in corpus
* average number of times each word is used in an article

In [ ]:
dtm_df = pd.DataFrame(tf_vectorizer.fit_transform(df_all.body.values.astype('U')).toarray(), columns=tf_vectorizer.get_feature_names(), index = df_all.index)

### Most common words in corpus + avg times each is used in an article 

In [ ]:
most_common_words = dtm_df.sum().sort_values(ascending=False)[0:500]
avg_times_used = dtm_df.mean().sort_values(ascending=False)[0:500]
df_top = pd.DataFrame(most_common_words)
df_top.columns = ['word_count']
df_top['avg_used'] = avg_times_used
df_top.to_csv('output/top_words_lq.csv', encoding='utf-8', index=True, header=True)

### To explore metadata for individual articles
For easy reference to the full metadata for each article contained in df_all using "article_jcode," which is the first column in article_lists_df.

In [ ]:
topic_10_1 = df_all[df_all['article_jcode'].str.contains('4309398')]
topic_10_1

## Alt: Create topic model viz
* Use pyLDA vis to visualize alternate topics generated via scikit-learn
* outputs to html file for future reference outside of notebook
* pyLDAvis is based on LDAvis (for R) and using "relevance" method for ranking terms within a topic

In [ ]:
pyLDAvis.enable_notebook()
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.save_html(p,'output/lda_tm_lq.html')